# Imports and Constants

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *


from_who = "our"   # 2 possibilities : "mohameds" or "our"
voxel_size = "2"
radius_mask = "7"
maps_folder="brain_maps/"+from_who+"_maps"+"_"+voxel_size
masks_folder="masks/"+from_who+"_masks"+"_"+voxel_size+"_radius="+radius_mask

SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000
class_labels = [""] * (2 * length_one_modality)
for i in range(length_one_modality * 2):
    class_labels[i] = classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length_one_modality])
labels["aud"] = np.array(class_labels[length_one_modality:])
labels_same = labels["vis"]

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                              (["vis"], ["PT_L", "PT_R"]),
                              (["aud"], ["V5_L", "V5_R"]),
                              (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                             (["vis", "aud"], ["PT_L", "PT_R"])]

# Analyses

In [ ]:
maps_masked, masks_exist = load_full_data(subjects_ids, length_one_modality, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"))

C_log_values = np.linspace(-5,5,num=200)
C_values = 10**C_log_values
kernels = ["linear","rbf","sigmoid","poly_3"]

In [ ]:
acc_group_combination = dict()
for kernel in kernels :
    acc_group_combination[kernel]=[None]*len(C_values)

start_time = time.time()
for i,C in enumerate(C_values) :
    for kernel_long in kernels :
        spl = kernel_long.split("_")
        kernel = spl[0]
        degree=3
        if len(spl) > 1:
            degree=int(spl[1])
        scaler = sklearn.preprocessing.StandardScaler()
        classifier = sklearn.svm.SVC(C = C, kernel = kernel, degree = degree, random_state=SEED)
        pipeline = Pipeline([('std_scaler', scaler), ('svm', classifier)])
        decoder = Decoder(n_perm=n_individual_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)
        decoder.set_masks_exist(masks_exist)

        # launch analyses
        confusion_matrixes_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
        group_confusion_matrixes_within = average_dicos(confusion_matrixes_within)

        confusion_matrixes_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
        group_confusion_matrixes_cross = average_dicos(confusion_matrixes_cross)

        # saving results
        params = classifier.get_params()
        out_directory = "out/"+from_who+str(classifier)+"/"
        create_directory(out_directory)

        save_dicts(out_directory + "confusion_matrixes_within.csv", confusion_matrixes_within,
                   list(confusion_matrixes_within[0].keys()), subjects_ids)
        acc_within = compute_metric(out_directory, subjects_ids, {'name': 'accuracy', 'function': accuracy}, "within", masks_exist, ret=True)

        save_dicts(out_directory + "confusion_matrixes_cross.csv", confusion_matrixes_cross,
                   list(confusion_matrixes_cross[0].keys()), subjects_ids)
        acc_cross = compute_metric(out_directory, subjects_ids, {'name': 'accuracy', 'function': accuracy}, "cross", masks_exist, ret=True)

        within_modality_group_results = average_dicos(acc_within)
        save_dicts(out_directory + "group_scores_within.csv", [within_modality_group_results],
                   list(within_modality_group_results.keys()), [0])
        acc_group_combination[kernel_long][i] = within_modality_group_results

        cross_modality_group_results = average_dicos(acc_cross)
        save_dicts(out_directory + "group_scores_cross.csv", [cross_modality_group_results],
                   list(cross_modality_group_results.keys()), [0])
        acc_group_combination[kernel_long][i].update(cross_modality_group_results)

    if (i+1)%10 == 0 :
        duration = time.time()-start_time
        print(str(i+1)+"/"+str(len(C_values))+" iterations done in "+str(duration)+" seconds")

# Plot scores depending on parameters

In [ ]:
import math
plt_directory = "plots/"+from_who+"_"+"tuning_parameters/"
create_directory(plt_directory)
plotter = Plotter(plt_directory, subjects_ids)

n_analyses = len(list(acc_group_combination["linear"][0].keys()))
fig, axs = plt.subplots(nrows=math.ceil(n_analyses/2),ncols=2,figsize=(15,25))

for i, analysis in enumerate(acc_group_combination["linear"][0]):
    # faire le plot
    for kernel_long in kernels :
        kernel_scores = [score[analysis] for score in acc_group_combination[kernel_long]]
        axs[math.floor(i/2),i%2].plot(C_log_values,kernel_scores,label=kernel_long)
        axs[math.floor(i/2),i%2].set_title(analysis)
    #plt.axvline(0, color="gray", alpha=0.5)
    #plotter.save(analysis+" - accuracy depending on C and kernel of SVM","","mean accuracy","log10(C)",legend="out")

plt.suptitle("Accuracy depending on C and kernel of SVM",fontsize=24)
plotter.save(analysis,"","mean accuracy","log10(C)",legend="best")